<a href="https://colab.research.google.com/github/mingd00/News-RAG/blob/main/news_api2json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import json
import re
from urllib.parse import urlparse
from email.utils import parsedate_to_datetime

In [2]:
def get_article_content(url: str) -> str:
    """
    주어진 URL에서 기사 본문을 추출하는 함수.

    :param url: 뉴스 기사 페이지의 URL
    :return: 기사 본문 텍스트 (사진은 제외됨)
    """
    try:
        # 웹 페이지 요청 보내기
        response = requests.get(url)

        # 요청이 성공적일 때
        if response.status_code == 200:
            # BeautifulSoup을 사용해 HTML 파싱
            soup = BeautifulSoup(response.text, 'html.parser')

            # 기사 본문이 들어있는 <article> 태그 찾기
            article = soup.find('article', {'id': 'dic_area', 'class': 'go_trans _article_content'})

            # 기사 본문 추출
            if article:
                content = article.get_text(strip=True)
                return content
            else:
                return "기사 본문을 찾을 수 없습니다."
        else:
            return f"웹사이트 요청 실패: {response.status_code}"

    except Exception as e:
        return f"오류 발생: {str(e)}"


# 사용 예시
url = 'https://n.news.naver.com/mnews/article/421/0008233058?sid=102'  # 여기에 실제 뉴스 기사 URL을 넣으세요
article_content = get_article_content(url)
print(article_content)

최경주가 2024년 5월15일 제주 서귀포 핀크스 콜프클럽에서 열린 한국프로골프투어(KPGA) SK텔레콤 채리티 오픈(자선경기)에서 10번홀 세컨 아이언샷을 하고 있다. (SK텔레콤 제공) 2024.5.15/뉴스1(제주=뉴스1) 고동명 기자 = 5월 한 달간 제주도내에서 총 22개 스포츠대회가 개최된다고 6일 제주도가 밝혔다.도에 따르면 국제대회로는 오는 24~25일 탑동공원과 성판악 일대에서 '제24회 제주 국제 울트라마라톤 대회'가 열린다. 이 대회에선 10개국 선수와 관계자 등 200여 명이 참가한다.전국대회로는 8일 'KPGA 클래식 골프대회'와 15일 'SK텔레콤2025 골프대회'가 연달아 개최된다. 전국에서 모인 프로골프선수 240여 명이 이들 대회에서 최강자를 가린다. 15일에는 '제80회 '제주 전국 학생테니스 선수권대회'도 예정돼 있다.도내 대회는 지난 3일 열린 제3회 '제주특별자치도체육회장배 전도파크골프대회'를 시작으로 4일 '2025 도족구협회장배 전도족구대회', 10일 '2025 제37회 미스터제주 챔피언십', 17일 '제17회 미디어제주기 전도 유소년 축구대회' 등이 도내 곳곳에서 펼쳐진다.이외에도 △2025 제주 챌린저컵 유소년 축구 리그 대회(3~4일, 외도구장) △제26회 KCTV배 제주도 배드민턴 대회(10~11일, 제주복합체육관) △제25회 도협회장배 전도여성게이트볼대회(24일, 제주복합체육관) 등 다양한 종목 대회가 열린다.장애인대회로는 9일 '제19회 제주도지사배 전국장애인사이클대회'를 시작으로 11일 '제3회 제주도장애인 수영연맹회장배 어울림수영대회', 15일 '제16회 전도시각장애인플라잉디스크대회 및 한·일 국제플라잉디스크교류대회', 31일 '제4회 제주시 어울림 생활체육대회'가 열린다.도는 이달 열리는 국제대회에선 11억 2600만 원, 전국대회는 80억 8000만 원, 도내대회는 3억 8200만 원 등의 지역경제 파급효과를 예상하고 있다.


In [ ]:
news_media_mapping = {
    "yonhapnews.co.kr": "연합뉴스",
    "yonhapnewstv.co.kr": "연합뉴스TV",
    "news1.kr": "뉴스1",
    "edu.donga.com": "동아일보",
    "biz.heraldcorp.com": "헤럴드경제",
    "daily.hankooki.com": "한국일보",
    "kmib.co.kr": "기독교일보",
    "kbs.co.kr": "KBS",
    "munhwa.com": "문화일보",
    "sports.naver.com": "네이버 스포츠",
    "newsis.com": "뉴시스",
    "segye.com": "세계일보",
    "hankooki.com": "한겨레",
    "chosun.com": "조선일보",
    "joongang.co.kr": "중앙일보",
    "khan.co.kr": "경향신문",
    "hani.co.kr": "한겨레",
    "sports.chosun.com": "스포츠조선",
    "sports.donga.com": "스포츠동아",
    "news.naver.com": "네이버 뉴스",
    "n.news.naver.com": "네이버 뉴스",
    "m.post.naver.com": "네이버 포스트",
    "seoul.co.kr": "서울신문",
    "etnews.com": "전자신문",
    "ytn.co.kr": "YTN",
    "tbs.seoul.kr": "TBS",
    "mbn.co.kr": "MBN 뉴스",
    "news.kmib.co.kr": "국민일보",
    "newdaily.co.kr": "뉴데일리",
    "yna.co.kr": "연합뉴스",
    "naeil.com": "내일신문",
    "kihoilbo.co.kr": "기호일보",
    "edaily.co.kr": "이데일리",
    "fnnews.com": "파이넨셜뉴스",
    "hankyung.com": "한경",
    "incheonnews.com": "인천뉴스",
    "bloter.net": "BROTER",
    "dt.co.kr": "디지털타임스",
    "sentv.co.kr": "서울경제TV" ,
    "econovill.com": "이코노믹 리뷰",
    "nytimes.com": "The New York Times",
    "digitaltoday.co.kr": "Digital Today",
    "koreajoongangdaily.joins.com": "Korea JoongAng Daily",
    "ddaily.co.kr": "디지털 데일리",
    "hankyung.com": "한국경제신문",
    "sedaily.com": "서울경제",
    "sportsseoul.com": "스포츠서울",
    "joongboo.com": "중부일보",
    "nocutnews.co.kr": "노컷뉴스",
    "imbc.com": "MBC 뉴스",
    "news.mt.co.kr": "머니투데이",
}

In [ ]:
# URL에서 도메인 추출 함수
def extract_domain(url):
    parsed_url = urlparse(url)
    domain = parsed_url.hostname
    if domain.startswith("www."):
        domain = domain[4:]  # "www." 제거
    return domain

In [ ]:
# 태그 제거
def clean_html_tags(text):
    return re.sub(r"<.*?>", "", text)

# 날짜 형식 지정
def format_date(pub_date_str):
    try:
        dt = parsedate_to_datetime(pub_date_str)
        return dt.strftime("%Y-%m-%d %H:%M")  # 예: 2025-05-06 10:59
    except Exception as e:
        print(f"날짜 파싱 오류: {e}")
        return pub_date_str

In [ ]:
# 네이버 API 키 (자신의 API 키로 교체하세요)
client_id = "gh33NVdUbiTCVrbFAQhV"
client_secret = "Wg6SoJlOj6"

# 뉴스 데이터를 담을 리스트
news_data = []

In [ ]:
# 요청 URL을 위한 기본 설정
base_url = "https://openapi.naver.com/v1/search/news.json"

# 네이버 API 요청 헤더 설정
headers = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret
}

In [ ]:
# 요청할 뉴스의 총 개수
page_offset = 2000  # 수동 조정(페이지 번호)

total_news_count = 1000 # 몇 개 가져올지
display_count = 100  # 한 번에 가져올 뉴스 수

# API에서 받아올 페이지 수 (예: 100 페이지 * 100개 = 10000개)
total_pages = total_news_count // display_count


# 여러 페이지의 뉴스 정보를 가져오는 함수
for start in range(1, total_pages + 1):
    params = {
        "query": "스포츠",  # 검색할 키워드
        "display": display_count,  # 한 번에 100개의 뉴스
        "start": page_offset + (start - 1) * display_count + 1,
        "sort": "sim"  # 유사도 기준 정렬
    }

    try:
        response = requests.get(base_url, headers=headers, params=params)
        data = response.json()

        # 각 기사에서 제목, 날짜, 링크, 내용 추출
        for item in data["items"]:
            # 링크가 네이버 뉴스 주소로 시작하는 경우에만 처리
            if item["link"].startswith("https://n.news.naver.com/"):
                news_item = {
                    "title": clean_html_tags(item["title"]),  # 제목
                    "date": format_date(item["pubDate"]),  # 날짜
                    "link": item["link"],  # 링크
                    "content": get_article_content(item["link"]),  # 기사 내용 (요약)
                    "journal": news_media_mapping.get(extract_domain(item["originallink"]), "Unknown")   # 언론사
                }
                news_data.append(news_item)

        # 요청 사이에 시간 지연을 주어 서버 부하 방지
        time.sleep(1)  # 1초의 딜레이를 주어 요청 속도를 조절

    except Exception as e:
        print(f"Error occurred while fetching data for page {start}: {e}")
        break

print(f'현재까지 수집된 기사 수: {len(news_data)}')

Error occurred while fetching data for page 1: 'items'
현재까지 수집된 기사 수: 221


In [ ]:
# 결과를 JSON 파일로 저장
with open("sports_news.json", "w", encoding="utf-8") as f:
    json.dump(news_data, f, ensure_ascii=False, indent=4)

print(f"총 {len(news_data)}개의 뉴스가 저장되었습니다.")

총 221개의 뉴스가 저장되었습니다.


In [5]:
# 저장된 JSON 파일 열기
with open("sports_news.json", "r", encoding="utf-8") as f:
    news_data = json.load(f)

# 첫 번째 뉴스의 content 확인
first_news_content = news_data[0]

print(first_news_content)

{'title': "코오롱스포츠, 서도호 작가 전시 '더 제네시스 익스비션' 참여", 'date': '2025-05-06 10:59', 'link': 'https://n.news.naver.com/mnews/article/001/0015370959?sid=101', 'content': '코오롱스포츠, 서도호 작가 전시 \'더 제네시스 익스비션\' 참여[코오롱인더스트리FnC부문 제공. 재판매 및 DB 금지](서울=연합뉴스) 강애란 기자 = 코오롱인더스트리FnC부문의 아웃도어 브랜드 코오롱스포츠는 세계적인 작가 서도호와 협업했다고 6일 밝혔다.서 작가는 우리나라를 대표하는 설치미술가이자 현대미술가로, 영국 런던을 기반으로 세계 무대에서 활동하고 있다.코오롱스포츠는 지난 1일부터 영국 런던 테이트 모던 미술관에서 연 서 작가의 대규모 전시회 \'더 제네시스 익스비션: 서도호:워크 더 하우스\'에 참여했다.코오롱스포츠가 협업한 작품은 \'브릿지 프로젝트\'의 \'S.O.S 수트\'로 코오롱스포츠의 기술력을 집약한 라이프텍 재킷을 재해석했다.서 작가는 집의 가장 작은 단위를 옷으로 가정하고 옷의 기능과 기술을 연구해 극한의 환경에서 생존할 수 있는 방법을 작품으로 승화시켰다. 기능적으로 생존복이지만 개념적으로는 완벽한 집이라는 상징성을 담았다.코오롱스포츠 관계자는 "이번 작품은 생존이라는 키워드 아래, 예술과 기술이 교차하는 지점을 구현했다"며 "앞으로도 다양한 분야와의 협업을 통해 브랜드 가치를 더욱 다층적으로 표현하고 사람과 환경, 기술을 잇는 브랜드로 나아가겠다"고 말했다.aeran@yna.co.kr', 'journal': '연합뉴스'}


In [10]:
# "journal" 값이 "Unknown"인 데이터만 추출
unknown_journal_data = [item for item in news_data if item['journal'] == 'Unknown']

# title만 한 줄씩 출력
for item in unknown_journal_data:
    print(item['title'])

교황 합성 사진 논란에…트럼프 &quot;아내는 귀엽다더라&quot;
트럼프 “교황 합성 사진은 장난…아내는 귀엽다고 해&quot;
트럼프, '교황 합성사진' 논란에 &quot;아내는 귀엽다고 하던데&quot;
“아내는 교황 합성사진 귀엽다던데”…트럼프 발언에 신자들 ‘부글부...
'교황 합성 사진 논란' 트럼프 &quot;아내는 귀엽다더라&quot;
트럼프, 교황 합성사진 논란에 &quot;아내가 귀엽다던데&quot;
애물단지서 스포츠메카로…용인 미르스타디움의 변신
[동정]김홍배 전 스포츠서울 출판기획팀 국장, 큰아들 결혼
인하대 스포츠파크 개장…지역사회에도 개방
LGU+, e스포츠 행사서 '와이파이7' 경쟁력 선보인다
스포츠 역베팅 사기사건 수사 속도…제주 센터장 2명 구속
인하대, 복합 스포츠 공간인 ‘인하 스포츠파크’ 개장
연예인 내세운 대형 스포츠센터 갑자기 폐업‥장기 회원권도 팔았는데
국내 시장 침체에도…'EV3' 덕에 전기차 판매는 더 늘었다
넥센타이어와 역동적인 모터스포츠 체험하세요
LG U+, e스포츠 행사서 4배 빠른 와이파이7 공유기 체험부스 운영
경찰, 스포츠 역베팅 제주 센터장 2명 구속
서울교육감배 학교스포츠클럽대회 10일 개막…9월까지
진주시,‘제64회 경상남도민체육대회’ 대장정 마무리
페라리, 모터스포츠 경험 담은 880마력의 ‘296 스페치알레’·’296 스페...
서울 학생 5만여명 참가하는 '학교스포츠클럽대회' 5월 개막
삼성라이온즈, 5월 포항 출격...프로야구 3연전 '포항' 달군다
제주경찰, 스포츠 역베팅 투자 사기 수사 속도…2명 구속
경과원, 서울국제가구전서 29억 상담 실적
복합스포츠문화공간 'SERI PAK with 용인' 5월 개관
인하대, 인하 스포츠파크 개장…기존 운동장 시설 개선
금호타이어 초고성능 퍼포먼스 '엑스타 스포츠' 출시
경기도, 46개 치매안심센터서...예방·진단·돌봄까지
서천군, 제3차 정책현답 추진…종합 스포츠단지 점검
[부고] 원영신(연세대 스포츠응용산업학과 명예교수)씨 별세 외
대전시교육청